<a href="https://colab.research.google.com/github/dpostolovski/eeg_is/blob/train_compare_full_data/Faza4-validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#@title Инсталирање и вчитување на потребните библиотеки

# Библиотека за истражување, визуелизација и анализирање на човечки 
# неврофизиолошки податоци (EEG, sEEG и др)
!pip install mne 
!pip install termcolor
!wget "https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/EEGModels.py"
!mkdir saved_models

%tensorflow_version 1.12.0

import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd
from termcolor import colored
import mne
from sklearn.decomposition import PCA, FastICA
import mne
from sklearn import metrics 
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint 
from sklearn.model_selection import train_test_split
from EEGModels import EEGNet,ShallowConvNet
import scipy.io as sio
from scipy.fft import fft
from scipy import signal
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras import backend as K
from tensorflow import keras
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from EEGModels import EEGNet,ShallowConvNet,DeepConvNet
from tensorflow.keras.optimizers import Adam

K.set_image_data_format('channels_first')

     |████████████████████████████████| 6.6MB 2.7MB/s 
--2020-06-13 22:46:40--  https://raw.githubusercontent.com/vlawhern/arl-eegmodels/master/EEGModels.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18283 (18K) [text/plain]
Saving to: ‘EEGModels.py’

EEGModels.py        100%[===================>]  17.85K  --.-KB/s    in 0.01s   

2020-06-13 22:46:40 (1.40 MB/s) - ‘EEGModels.py’ saved [18283/18283]

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.12.0`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [0]:
def reshape(data, labels, events, targets):
  mne_array = np.swapaxes(data, 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)

  labels_arr = labels.astype(np.int)
  return mne_array, labels_arr-1

In [29]:
# Иницијализација на променливите каде ќе бидат вчитани податоците
data = []
labels = []
events = []
targets = []

for i in range(1, 2): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  train_data = np.empty(0)
  train_labels = np.empty(0)
  train_events = np.empty(0)
  train_targets = np.empty(0)

  test_data = np.empty(0)
  test_labels = np.empty(0)
  test_events = np.empty(0)
  test_targets = np.empty(0)
  
  for j in range(4, 7): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') + '/Train'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Test_Data/' + file_name + "/" + file_train_set + "/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if train_data.size != 0:
      train_data = np.concatenate((train_data, temp), axis=2)
    else: 
      train_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Test_Data/' + file_name + "/" + file_train_set + "/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if train_labels.size != 0:
        train_labels = np.concatenate((train_labels, temp))
      else:
        train_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Test_Data/' + file_name + "/" + file_train_set + "/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if train_events.size != 0:
        train_events = np.append(train_events, temp)
      else:
        train_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Test_Data/' + file_name + "/" + file_train_set + "/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if train_targets.size != 0:
        train_targets = np.concatenate((train_targets, temp))
      else:
        train_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(7, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02')

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Test_Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if test_data.size != 0:
      test_data = np.concatenate((test_data, temp), axis=2)
    else: 
      test_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Test_Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if test_labels.size != 0:
        test_labels = np.concatenate((test_labels, temp))
      else:
        test_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Test_Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if test_events.size != 0:
        test_events = np.append(test_events, temp)
      else:
        test_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Test_Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if test_targets.size != 0:
        test_targets = np.concatenate((test_targets, temp))
      else:
        test_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")
      
  # Зачувај ги податоците вчитани од испитниот примерок во низа


  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")



  #data = target_events_data_scaled
  X_train, y_train = reshape(train_data,train_labels,train_events, train_targets)
  X_test, y_test = reshape(test_data,test_labels,test_events, test_targets)


 # X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model.evaluate(X_test, y_test, verbose=1)
  print(score)

  predictions = model.predict(X_test)
  np.savetxt("predictions.csv", predictions, delimiter=",")
  np.savetxt("y_test.csv", y_test, delimiter=",")


	 - Податоците од сесија 4 се вчитани.
	 - Податоците од сесија 5 се вчитани.
	 - Податоците од сесија 6 се вчитани.
	 - Податоците од сесија 7 се вчитани.
Податоците од испитниот примерок 1 се вчитани.
Train on 4800 samples, validate on 1600 samples
Epoch 1/400
4600/4800 [===========================>..] - ETA: 0s - loss: 2.3152 - acc: 0.1372
Epoch 00001: val_loss improved from inf to 2.49190, saving model to saved_models/weights.best.basic_mlp.hdf5
4800/4800 [==============================] - 2s 448us/sample - loss: 2.3153 - acc: 0.1358 - val_loss: 2.4919 - val_acc: 0.1006
Epoch 2/400
4500/4800 [===========================>..] - ETA: 0s - loss: 2.2214 - acc: 0.1507
Epoch 00002: val_loss did not improve from 2.49190
4800/4800 [==============================] - 1s 121us/sample - loss: 2.2164 - acc: 0.1525 - val_loss: 2.5418 - val_acc: 0.0425
Epoch 3/400
4500/4800 [===========================>..] - ETA: 0s - loss: 2.1848 - acc: 0.1682
Epoch 00003: val_loss improved from 2.49190 to 2.4884

In [30]:
from collections import Counter
int_pred = np.argmax(predictions,axis=1)
int_ytest = np.argmax(y_test,axis=1)

correct = 0
for i in range(20):
  block_y = int_ytest[i*80:(i+1)*80]
  block_y_pred = int_pred[i*80:(i+1)*80]

  class_y = np.bincount(block_y).argmax()
  class_y_pred = np.bincount(block_y_pred).argmax()

  print(f"Class Y: {class_y}, prediciton: {class_y_pred}")
  if class_y == class_y_pred:
    correct = correct+1

correct

Class Y: 7, prediciton: 4
Class Y: 1, prediciton: 4
Class Y: 4, prediciton: 4
Class Y: 4, prediciton: 4
Class Y: 3, prediciton: 4
Class Y: 4, prediciton: 4
Class Y: 5, prediciton: 4
Class Y: 0, prediciton: 7
Class Y: 5, prediciton: 5
Class Y: 7, prediciton: 1
Class Y: 2, prediciton: 6
Class Y: 2, prediciton: 6
Class Y: 3, prediciton: 4
Class Y: 5, prediciton: 4
Class Y: 1, prediciton: 4
Class Y: 1, prediciton: 4
Class Y: 1, prediciton: 4
Class Y: 2, prediciton: 5
Class Y: 0, prediciton: 0
Class Y: 5, prediciton: 4


5